<a href="https://colab.research.google.com/github/CygnusST3RN/FHE.0/blob/ankita_ml/FHE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [7]:
def read_dataset(url,col,scale):
  df=pd.read_csv(url)
  df.drop("Unnamed: 32", axis=1, inplace=True)
  X = df.iloc[:, col+1:]
  col_name=df.columns[col]
  y = df[col_name]
  for i in range (0,len(df)):
    if y[i]=='M':
      y[i]=1
    else:
      y[i]=0
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = MinMaxScaler(feature_range=(0, scale))
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [8]:
X_train, X_test, y_train, y_test = read_dataset('data.csv',1,1)

(143, 30)


<ipython-input-7-080874c512ac>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
<ipython-input-7-080874c512ac>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=16)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/10
27/27 [==============================] - 1s 3ms/step - loss: 0.6366 - accuracy: 0.7066
Epoch 2/10
27/27 [==============================] - 0s 3ms/step - loss: 0.6253 - accuracy: 0.7418
Epoch 3/10
27/27 [==============================] - 0s 3ms/step - loss: 0.6153 - accuracy: 0.7793
Epoch 4/10
27/27 [==============================] - 0s 3ms/step - loss: 0.6066 - accuracy: 0.8052
Epoch 5/10
27/27 [==============================] - 0s 3ms/step - loss: 0.5974 - accuracy: 0.8075
Epoch 6/10
27/27 [==============================] - 0s 3ms/step - loss: 0.5891 - accuracy: 0.8192
Epoch 7/10
27/27 [==============================] - 0s 3ms/step - loss: 0.5811 - accuracy: 0.8357
Epoch 8/10
27/27 [==============================] - 0s 3ms/step - loss: 0.5733 - accuracy: 0.8404
Epoch 9/10
27/27 [==============================] - 0s 3ms/step - loss: 0.5659 - accuracy: 0.8545
Epoch 10/10
27/27 [==============================] - 0s 3ms/step - loss: 0.5585 - accuracy: 0.8545
Test Accuracy: 82.5

In [37]:
def create_model_weights_file(model,scale):
  i=0
  for layer in model.layers:
    w = layer.get_weights()[0]
    b = layer.get_weights()[1]
    w=np.round_(w*scale)
    b=np.round_(b*scale)
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [38]:
create_model_weights_file(model,10)